# Chapter 4: Transforming vectors and graphics

**NOTE** For most of the examples in this chapter, the code is in other .py files in this directory, rather than directly in this Notebook.  You can run the other files in the notebook below, and a PyGame window will open up.  For instance `!python script.py` runs `script.py` as if you're running it from the command line. When you're done admiring the graphics the scripts create, you can close the window and the Jupyter cell will finish computing.

Make sure to follow the instructions from Appendix C before beginning this chapter, including

`pip install PyGame`

`pip install PyOpenGL`

# 4.1 Transforming 3D objects

## 4.1.1 Drawing a transformed object

symbol **`!`** tells jypiter notebook to execute script in command lie 

In [1]:
!python draw_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
!python scale_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
!python scale_translate_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## 4.1.2 Composing vector transformations

When function ends with `()` -- *parentheses*, it means that function will be executed  
* [Programming Terms: First-Class Functions](https://www.youtube.com/watch?v=kr0mpwqttM0)  
* [Currying - Pycon Africa](https://www.youtube.com/watch?v=L3G7BH7n0K0&list=LL4vFTVTAJuWCzJysoHgHzkA&index=3)  
* [Currying - python-course](https://www.python-course.eu/currying_in_python.php#General-Idea)  
* [Currying in Python](https://mtomassoli.wordpress.com/2012/03/18/currying-in-python/)  

We can write a general-purpose compose function that takes two Python functions (vector transformations for instance) and returns a new function, which is their composition:

In [1]:
def compose(f1,f2):
    def new_function(input):
        return f1(f2(input))
    return new_function

```
def scale2_then_translate1left(v):    
      return translate1left(scale2(v))
```
      
Instead of defining scale2_then_translate1left as its own function, we could write:  

`scale2_then_translate1left = compose(translate1left, scale2)`

this is called *composition of function*

**The composition of two functions is a chaining process in which the output of the inner function becomes the input of the outer function.**

In [2]:
def square(x):
    return x**2
def adding(x):
    return x+x

In [3]:
square_adding = compose(square, adding)
square_adding(1)

4

In [4]:
from vectors import scale 

def scale2(v): 
    return scale(2.0, v)

In [5]:
def polygon_map(transformation, polygons):
    return [
        [transformation(vertex) for vertex in triangle]
        for triangle in polygons
    ]

In [6]:
from teapot import load_triangles

polygon_map(scale2, load_triangles())[0]

[(1.4, 1.0521480000000003, 0.0),
 (1.2734839999999998, 1.124108, -0.541836),
 (1.2915, 1.0521480000000003, -0.5495)]

In [7]:
from draw_model import draw_model

#draw_model(polygon_map(scale2, load_triangles()))

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
def scale_by(scalar):
    def new_function(v):
        return scale(scalar, v)
    return new_function

In [9]:
scale2 = scale_by(2)
scale2((1,1,1))

(2, 2, 2)

In [10]:
scale_by(2)((1,1,1))

(2, 2, 2)

In [11]:
from vectors import add

def translate_by(xform):
    def new_function(v):
        return add((xform), (v))
    return new_function

In [12]:
translate2 = translate_by((0,5,0))
translate2((1,1,1))

(1, 6, 1)

In [13]:
translate_by((0,5,0))((1,1,1))

(1, 6, 1)

## 4.1.3 Rotating an object about an axis

You already saw how to do rotations in 2D in chapter 2: you convert the Cartesian coordinates to polar coordinates, increase or decrease the angle by the rotation factor, and then convert back.

In [14]:
def rotate2d(angle, vector): 
    l,a = to_polar(vector)
    return to_cartesian((l, a+angle))

In [15]:
!python rotate_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
def rotate_z(angle, vector): 
    x,y,z = vector 
    new_x, new_y = rotate2d(angle, (x,y)) 
    return new_x, new_y, z

# currying rotate_z()
def rotate_z_by(angle):
    def new_function(vector):
        return rotate_z(angle, vector)
    return new_function

In [17]:
from math import *
from transforms import to_polar, to_cartesian

#draw_model(polygon_map(rotate_z_by(pi/4.), load_triangles()))

In [18]:
from functools import partial

rot_z = partial(rotate_z, pi/2.)

#draw_model(polygon_map(rot_z, load_triangles()))

In [19]:
def rotate_x(angle, vector):
    x,y,z = vector
    new_y, new_z = rotate2d(angle, (y,z))
    return x, new_y, new_z

# currying rotate_x()
def rotate_x_by(angle):
    def new_func(vector):
        return rotate_x(angle, vector)
    return new_func

In [20]:
#draw_model(polygon_map(rotate_x_by(pi/2.), load_triangles()))

!python rotate_teapot_x.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## 4.1.4 Inventing your own geometric transformations

In [21]:
def strech_x(vector):
    x,y,z = vector
    return (4.*x, y, z)

In [22]:
!python stretch_teapot.py

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [23]:
def stretch_y(vector):
    x,y,z = vector
    return (x, 2.*y, z)

#draw_model(polygon_map(stretch_y, load_triangles()))

In [24]:
def stretch_z(vector):
    x,y,z = vector
    return (x, y*y*y, z)

#draw_model(polygon_map(stretch_z, load_triangles()))

In [25]:
def slant_xy(vector):
    x,y,z = vector
    return (x+y, y, z)

#draw_model(polygon_map(slant_xy, load_triangles()))

## 4.1.5 Exercises

**Exercise 4.1:** Implement a `translate_by` function referred to in section 4.1.2, taking a translation vector as an input and returning a translation function as an output.

In [26]:
def translate_by(vector1):
    def new_function(vector2):
        return add(vector1, vector2)
    return new_function

In [27]:
translate_by_1 = translate_by((1,1,1))

In [28]:
#draw_model(polygon_map(translate_by_1, load_triangles()))

**Exercise 4.6:** Modify the `compose(f,g)` function to `compose(*args)`, which takes several functions as arguments and returns a new function that is their composition.

In [29]:
def adding(*args):
    print(args) #original
    print(*args) #unpacked
    return sum(args)

adding(4,3,2,1)

(4, 3, 2, 1)
4 3 2 1


10

In [30]:
def compose(*args):
    def new_function(input):
        state = input
        for f in reversed(args):
            state = f(state)
        return state
    return new_function

In [31]:
scale_add = compose(scale_by(2), translate_by((1,1,1)))
scale_add((2,2,2))

(6, 6, 6)

In [32]:
def prepend(string):
    def new_function(input):
        return string + input
    return new_function

In [33]:
f = compose(prepend('P'), prepend('y'), prepend('t'))
f('hon')

'Python'

**Exercise 4.2:** Render the teapot translated by 20 units in the negative z direction. What does the resulting image look like?


In [34]:
translate_z = translate_by((0,0,-20))
#draw_model(polygon_map(translate_z, load_triangles()))

**Mini-project 4.3:** What happens to the teapot when you scale every vector by a scalar between 0 and 1? What happens when you scale it by a factor of -1?

In [35]:
#draw_model(polygon_map(scale_by(0.5), load_triangles()))

In [36]:
#draw_model(polygon_map(scale_by(-1), load_triangles()))

**Exercise 4.4:** First apply translate1left to the teapot and then apply scale2. How is the result different from the opposite order of composition? Why?

In [37]:
trans_scale = compose(translate_by((-1,0,0)), scale_by(2))
#draw_model(polygon_map(trans_scale, load_triangles()))

In [38]:
scale_trans = compose(scale_by(2), translate_by((-1,0,0)))
#draw_model(polygon_map((scale_trans), load_triangles()))

**Exercise 4.7:** Write a curry2(f) function that takes a Python function f(x,y) with two arguments and returns a curried version. For instance, once you write g = curry2(f), the two expressions f(x,y) and g(x)(y) should return the same result.

In [39]:
def curry2(f):
    def new_function1(input1):
        def new_function2(input2):
            return f(input1, input2)
        return new_function2
    return new_function1

In [40]:
r = (1,0,0)
s = 5

scale_curried = curry2(scale)
scale_curried(s)(r), scale(s,r)

((5, 0, 0), (5, 0, 0))

In [41]:
scale_by = curry2(scale)
scale_by(2)((1,2,3))

(2, 4, 6)

**Exercise 4.8:** Without running it, what is the result of applying the transformation compose(rotate_z_by(pi/2),rotate_x_by(pi/2))? What if you switch the order of the composition?

In [74]:
#draw_model(polygon_map(rotate_z_by(pi/2), load_triangles()))
#draw_model(polygon_map(rotate_x_by(pi/2), load_triangles()))
#draw_model(polygon_map(compose(rotate_z_by(pi/2), rotate_x_by(pi/2)), load_triangles()))
#draw_model(polygon_map(compose(rotate_x_by(pi/2), rotate_z_by(pi/2)), load_triangles()))

**Exercise 4.9:** Write a function stretch_x(scalar,vector) that scales the target vector by the given factor but only in the x direction. Also write a curried version stretch_x_by so that stretch_x_by(scalar)(vector) returns the same result.

In [78]:
def stretch_x(scalar, vector):
    x,y,z = vector
    return x*scalar, y, z

In [79]:
stretch_x(5, (1,1,1))

(5, 1, 1)

In [84]:
def strecth_x_by(scalar):
    def new_function(vector):
        return stretch_x(scalar, vector)
    return new_function

In [86]:
strecth_x_by(5)((1,1,1))

(5, 1, 1)